### Libraries

In [1]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import features
from rasterstats import zonal_stats
from glob import glob
from osgeo import gdal
import os
import json

### Folder and data paths

In [2]:
# Current working directory
cwd = os.getcwd()

# Folder paths
image_folder = os.path.join(cwd, "Data", "Imagery")  # Folder containing imagery data
mosaic_folder = os.path.join(
    image_folder, "Mosaic"
)  # Folder that will contain the mosaic imagery

# Create the mosaic_folder if it doesn't exist
os.makedirs(mosaic_folder, exist_ok=True)

# List of image files in the image_folder
image_list = glob(os.path.join(image_folder, "*.tif"), recursive=True)

# Path to the shapefile
shp_path = os.path.join(cwd, "Data", "Sites", "ICOAST_midwest.shp")

### Mosaic

In [3]:
# List of image files to be mosaicked
files_to_mosaic = image_list

# Enable GDAL exceptions handling
gdal.UseExceptions()

# Perform mosaic operation using GDAL
# Create a new raster mosaic file named 'Sentinel_mosaic.tif' in the mosaic_folder
# Warp the listed image files to the new mosaic
res = gdal.Warp(
    os.path.join(mosaic_folder, "Sentinel_mosaic.tif"),
    files_to_mosaic,
    srcSRS="EPSG:4326",
    dstSRS="EPSG:4326",
)

# Close the GDAL raster dataset to release resources
res = None

### Mean band differences

In [4]:
# Open the raster stack (mosaic)
rgb_stack = rasterio.open(os.path.join(mosaic_folder, "Sentinel_mosaic.tif"))

# Extract metadata from the RGB stack
profile = rgb_stack.meta

# Update metadata to define data type, number of bands, compression, and nodata value
profile.update(dtype=rasterio.float64, count=1, compress="lzw", nodata=-9999)

# Read specific bands (blue band and NIR band) from the RGB stack
b2 = rgb_stack.read(2)
b8 = rgb_stack.read(8)

# Calculate the absolute difference between band 8 and band 2
difference = abs(b8 - b2)

# Define output directory for difference calculation
difference_out = os.path.join(mosaic_folder, "Zonal stats")
os.makedirs(difference_out, exist_ok=True)

# Write the difference raster to a file
with rasterio.open(
    os.path.join(difference_out, "difference.tif"), "w", **profile
) as dst:
    dst.write_band(1, difference)

# Perform zonal statistics using the shapefile and the difference raster
zonal = zonal_stats(
    shp_path,
    os.path.join(difference_out, "difference.tif"),
    stats="mean",
    geojson_out=True,
)

# Prepare GeoJSON output
result = {"type": "FeatureCollection", "features": zonal}

# Write zonal statistics result to a GeoJSON file
with open(os.path.join(mosaic_folder, "Zonal stats", "zonal.geojson"), "w") as outfile:
    json.dump(result, outfile)

# Read GeoJSON file into a GeoDataFrame
zonal_gjson = gpd.read_file(os.path.join(mosaic_folder, "Zonal stats", "zonal.geojson"))

# Write GeoDataFrame to a shapefile
zonal_gjson.to_file(os.path.join(mosaic_folder, "Zonal stats", "mu_difference.shp"))

# Read the shapefile into a GeoDataFrame
zonal_gpd = gpd.read_file(
    os.path.join(mosaic_folder, "Zonal stats", "mu_difference.shp")
)

### Sigma calculation

In [5]:
# Create a generator expression to iterate over geometry and values from the GeoDataFrame
geom_value = (
    (geom, value) for geom, value in zip(zonal_gpd.geometry, zonal_gpd["mean"])
)

# Rasterize the geometries and values into a raster image
rasterized = features.rasterize(
    geom_value,
    out_shape=rgb_stack.shape,
    transform=rgb_stack.transform,
    all_touched=True,
    fill=-9999,  # Background value
    dtype=np.float64,
)

sigma_folder = os.path.join(
    mosaic_folder, "Sigma"
)  # Folder that will contain the sigma imagery
os.makedirs(sigma_folder, exist_ok=True)


# Create a new raster file for Sigma values
with rasterio.open(
    os.path.join(mosaic_folder, "Sigma", "Sigma.tif"),
    "w",
    driver="GTiff",
    transform=rgb_stack.transform,
    dtype=rasterio.float64,
    count=1,
    width=rgb_stack.width,
    height=rgb_stack.height,
) as dst:
    # Write the rasterized data to the file
    dst.write(rasterized, indexes=1)

In [ ]:
### kNDAVI calculation

In [6]:
# Open Sigma raster
sigma_raster = rasterio.open(
    os.path.join(mosaic_folder, "Sigma", "Sigma.tif"), nodata=-9999
)
sigma = sigma_raster.read(1)

# Calculate knr
with np.errstate(divide="ignore", invalid="ignore"):
    knr = np.exp(-((b8 - b2) ** 2) / (2 * sigma**2))

kndavi = (1 - knr) / (1 + knr)

# Define output folder
kndavi_out = os.path.join(mosaic_folder, "kNDAVI")
os.makedirs(kndavi_out, exist_ok=True)

profile.update(nodata=0)

# Write kndavi to raster file
with rasterio.open(os.path.join(kndavi_out, "kndavi.tif"), "w", **profile) as dst:
    dst.write_band(1, kndavi)

# Close the raster files
sigma_raster.close()
rgb_stack = None